In [ ]:
import os
import pandas as pd

# This file automates the entire pipeline for assertion generation with chatgpt

## Step 1) Get Asserted Code From Github

### Step 1.1) Clean and process the code
### Step 1.2) Extract Ground-Truth Assertions & Relevant Statistics

In [ ]:
query_string="""SELECT f.repo_name, c.content
FROM `bigquery-public-data.github_repos.files` AS f
JOIN `bigquery-public-data.github_repos.contents` AS c
ON f.id = c.id
WHERE
NOT c.binary
AND f.path LIKE '%.py'
AND REGEXP_CONTAINS(c.content, r'(?m)^\s*assert ')
LIMIT """

def get_asserted_code(num=100000, verbose=False):
    query_string += str(num)
    
    if verbose:
        print("Running Query:")
        print(query_string)
        print()
    client = bq.Client()
    df = (
        client.query(query_string)
        .result()
        .to_dataframe(
            create_bqstorage_client=True,
        )
    )
    
    df.drop_duplicates(subset=["content"], keep="first", inplace=True)
    return df

## Step 2) Generate LLM Prompt & Query a GPT

## Step 3) Collect & Evaluate GPT's Response